In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

# data
import pandas as pd
import numpy as np

# viz
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
SQL_USERNAME = "postgres"
SQL_PASSWORD = "postgres" # change this
SQL_IP = "localhost"
PORT = 5432
DATABASE = "crowdfunding" # change this

In [3]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

In [4]:
# INSPECT

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

contacts_df
-----------
contact_id INTEGER
first_name VARCHAR(250)
last_name VARCHAR(250)
email VARCHAR(250)

campaign_df
-----------
cf_id INTEGER
contact_id INTEGER
company_name VARCHAR(250)
description VARCHAR(250)
goal DOUBLE PRECISION
pledged DOUBLE PRECISION
outcome VARCHAR(250)
backers_count INTEGER
country VARCHAR(250)
currency VARCHAR(10)
launch_date TIMESTAMP
end_date TIMESTAMP
category_id VARCHAR(50)
subcategory_id VARCHAR(250)

category_df
-----------
category_id VARCHAR(50)
category VARCHAR(250)

subcategory_df
-----------
subcategory_id VARCHAR(50)
subcategory VARCHAR(250)



In [5]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(autoload_with=engine)

# Print all of the classes mapped to the Base
print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.subcategory_df'>, <class 'sqlalchemy.ext.automap.category_df'>, <class 'sqlalchemy.ext.automap.contacts_df'>, <class 'sqlalchemy.ext.automap.campaign_df'>]


In [98]:
# join category.csv and subcategory.csv


join_query = """

SELECT 
  campaign_df.*,
  category_df.category,
  subcategory_df.subcategory

FROM campaign_df
LEFT JOIN category_df ON
category_df.category_id=campaign_df.category_id

LEFT JOIN subcategory_df ON
subcategory_df.subcategory_id=campaign_df.subcategory_id;


;"""

join_df = pd.read_sql(text(join_query), con=engine)

join_df.head(10)

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launch_date,end_date,category_id,subcategory_id,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,1970-01-01 00:00:01.581574,1970-01-01 00:00:01.614578,cat1,subcat1,food,food trucks
1,1175,2288,Werner-Bryant,Virtual uniform frame,1800.0,7991.0,successful,222,US,USD,1970-01-01 00:00:01.592629,1970-01-01 00:00:01.611986,cat1,subcat1,food,food trucks
2,873,2067,Stewart LLC,Cloned bi-directional architecture,1300.0,12047.0,successful,113,US,USD,1970-01-01 00:00:01.606630,1970-01-01 00:00:01.623388,cat1,subcat1,food,food trucks
3,2568,5989,Castillo-Carey,Cross-platform solution-oriented process impro...,142400.0,21307.0,failed,296,US,USD,1970-01-01 00:00:01.588655,1970-01-01 00:00:01.622351,cat1,subcat1,food,food trucks
4,1211,3307,"Wright, Hartman and Yu",User-friendly tertiary array,3300.0,12437.0,successful,131,US,USD,1970-01-01 00:00:01.611900,1970-01-01 00:00:01.613196,cat1,subcat1,food,food trucks
5,990,3489,Morgan-Martinez,Mandatory tertiary implementation,148500.0,4756.0,canceled,55,AU,AUD,1970-01-01 00:00:01.616476,1970-01-01 00:00:01.638770,cat1,subcat1,food,food trucks
6,2228,4404,Alexander-Williams,Triple-buffered cohesive structure,7200.0,6927.0,failed,210,US,USD,1970-01-01 00:00:01.618636,1970-01-01 00:00:01.625720,cat1,subcat1,food,food trucks
7,509,1867,Mcknight-Freeman,Upgradable scalable methodology,8300.0,6543.0,canceled,82,US,USD,1970-01-01 00:00:01.597295,1970-01-01 00:00:01.619068,cat1,subcat1,food,food trucks
8,966,4840,Huff LLC,Face-to-face clear-thinking Local Area Network,121500.0,119830.0,failed,2179,US,USD,1970-01-01 00:00:01.618031,1970-01-01 00:00:01.633064,cat1,subcat1,food,food trucks
9,2765,5565,Ho-Harris,Versatile cohesive encoding,7300.0,717.0,failed,10,US,USD,1970-01-01 00:00:01.606802,1970-01-01 00:00:01.632805,cat1,subcat1,food,food trucks


In [105]:

outcome = join_df.groupby(['outcome','category','subcategory'])['outcome'].count().reset_index(name="count")
display(outcome)


,outcome,category,subcategory,count
0,canceled,film & video,animation,1
1,canceled,film & video,documentary,4
2,canceled,film & video,drama,2
3,canceled,film & video,shorts,1
4,canceled,film & video,television,3
...,...,...,...,...
67,successful,publishing,radio & podcasts,4
68,successful,publishing,translations,14
69,successful,technology,wearables,28
70,successful,technology,web,36


### Treemap

In [83]:
import plotly.express as px

In [106]:
fig=px.treemap(outcome, path=['outcome','category','subcategory'], values='count',color='count')
fig.update_layout(
    margin=dict(l=10,r=10,t=50,b=10),
    autosize=False,
    height=800,
    width=1500,
    title=("Outcome grouped by category and subcategory")
)
fig.show()